In [ ]:
import numpy as np
import pandas as pd
import random as r
%matplotlib inline
import matplotlib.pyplot as plt # side-stepping mpl backend
from scipy.stats import entropy
pipe = subprocess.PIPE

def run(neurons=200, time=1000, rec=10, tmin=3, tmax=10):
    result = subprocess.run(
        ['go', 'run', 'main.go',
         f'-neurons={neurons}',
         f'-time={time}',
         f'-tmin={tmin}',
         f'-rec={rec}',
         f'-tmax={tmax}',
                            ], cwd='perf', stdout=pipe, stderr=pipe)

    if result.returncode==0:
        print(result.stdout.decode('utf-8'))
    else:
        print(result.stderr.decode('utf-8'))
        raise ValueError("Failure")

def render_output():
    df = pd.read_table('perf/dump.tsv')

    O=df.sum(axis=1)

    print(f'entropy: {entropy(O)}')

    fig, axs = plt.subplots(2, figsize = (20,10), sharex=True)
    axs[1].autoscale(enable=True, axis='x', tight=True)
    axs[0].plot(O)
    axs[1].imshow(df.T, interpolation='none', aspect='auto')
    fig.tight_layout()
    plt.show()

In [ ]:
run(neurons=400, time=20000, tmin=2, tmax=7, rec=4)
render_output()

In [ ]:
from skopt import Optimizer

opt = Optimizer([(1,4),(1,10),(1,10)])

def f(tmin, tmax, rec):
    run(neurons=400, time=20000, tmin=tmin, tmax=tmax, rec=rec)
    df = pd.read_table('perf/dump.tsv')

    O=df.sum(axis=1)

    return entropy(O)

for i in range(20):
    suggested = opt.ask()
    y = f(*suggested)
    opt.tell(suggested, y)
    print('iteration:', i, suggested, y)

In [ ]:
render_output()